In [148]:
import pandas as pd
import numpy as np
import hijri_converter


In [159]:
from hijri_converter import convert
def convert_date(date_string):
    return pd.to_datetime(date_string, format='%d/%m/%Y')

df = pd.read_excel("RequiredData.xlsx", sheet_name="Reservation List", converters={'Reservation Check-out Date ':convert_date  , 'Cancellation Date':convert_date , 'Reservation Check-in Date':convert_date ,'Reservation Creation Date (MM-DD-YYYY)':convert_date})

df['Hijri Check-out Date'] = df['Reservation Check-out Date '].apply(lambda x: convert.Gregorian(x.year, x.month, x.day).to_hijri())
df['Hijri Check-in Date'] = df['Reservation Check-in Date'].apply(lambda x: convert.Gregorian(x.year, x.month, x.day).to_hijri())


df.sort_values('Reservation Creation Date (MM-DD-YYYY)', inplace=True)
df.head(10)

,Reservation ID,Reservation Creation Date (MM-DD-YYYY),Reservation Check-in Date,Reservation Check-out Date,Reservation Total Price,#Rooms,Cancellation Date,"Type\nallotment (1), sales allotment (2), direct hotel (3)or direct agent(4)",Hijri Check-out Date,Hijri Check-in Date
0,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1,1444-10-22,1444-10-11
1,2,2023-01-01,2023-05-04,2023-05-09,5000,5,NaT,1,1444-10-19,1444-10-14
2,3,2023-01-01,2023-08-09,2023-08-10,200,5,2023-01-03,1,1445-01-23,1445-01-22
3,4,2023-01-02,2023-09-07,2023-09-16,7200,4,2023-01-03,1,1445-03-01,1445-02-22
4,5,2023-01-02,2023-01-09,2023-01-12,81400,20,NaT,2,1444-06-19,1444-06-16
5,6,2023-01-03,2023-04-21,2023-04-27,74243,55,NaT,1,1444-10-07,1444-10-01


In [150]:
from hijri_converter import Hijri, Gregorian
Hijri(1433, 9, 1)

Hijri(1433, 9, 1)

In [151]:
value_1 = df.loc[0,'Hijri Check-out Date']
value_2 = df.loc[5,'Hijri Check-out Date']


if value_1 > value_2:
    print("The first row is bigger.")
elif value_1 < value_2:
    print("The first row is smaller.")
else:
    print("The values are equal.")

The first row is bigger.


In [152]:
# Create a new DataFrame to store the filtered reservations
filtered_df = pd.DataFrame(columns=df.columns)


In [153]:
target_start_day_hijri = Hijri(1442,1,1) # Hijri date in the format 'yyyy-mm-dd'
target_end_day_hijri  = Hijri(1446,12,27) # Hijri date in the format 'yyyy-mm-dd'

In [154]:
def increment_hijri(hijri_date):
    dt_greg = hijri_date.to_gregorian().ctime()
    dt_greg = pd.to_datetime(dt_greg)
    dt_greg += pd.DateOffset(days=1)
    incremented_hijri_date = Gregorian(dt_greg.year, dt_greg.month, dt_greg.day).to_hijri()
    return incremented_hijri_date

In [155]:
# Iterate over each row in the original DataFrame
for index, row in df.iterrows():
    # Extract the check-in and check-out  Hijri dates 
    check_in_date_hijri = row['Hijri Check-in Date']
    check_out_date_hijri = row['Hijri Check-out Date']
    target_start_day_hijri_temp = Hijri(check_in_date_hijri.year, target_start_day_hijri.month, target_start_day_hijri.day) 
    target_end_day_hijri_temp = Hijri(check_out_date_hijri.year, target_end_day_hijri.month, target_end_day_hijri.day)



# Create a list to store the rows for each day within the range
    rows = []

    # Iterate over each day within the range
    current_day = check_in_date_hijri
    while current_day <= check_out_date_hijri:
        # Check if the current day falls within the check-in and check-out dates
        if target_start_day_hijri_temp <= current_day <= target_end_day_hijri_temp:
            # Append the row with the current day to the list
            rows.append(list(row)+[current_day])

        current_day = increment_hijri(current_day)

    if (rows == []):
        continue

    # Create a DataFrame for the rows for this reservation
    reservation_df = pd.DataFrame(rows, columns=list(df.columns)+['Target Date'])

    # Concatenate the reservation DataFrame with the filtered DataFrame
    filtered_df = pd.concat([filtered_df, reservation_df], ignore_index=True)



In [160]:
filtered_df

,Reservation ID,Reservation Creation Date (MM-DD-YYYY),Reservation Check-in Date,Reservation Check-out Date,Reservation Total Price,#Rooms,Cancellation Date,"Type\nallotment (1), sales allotment (2), direct hotel (3)or direct agent(4)",Hijri Check-out Date,Hijri Check-in Date,Target Date
0,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1,1444-10-22,1444-10-11,1444-10-11
1,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1,1444-10-22,1444-10-11,1444-10-12
2,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1,1444-10-22,1444-10-11,1444-10-13
3,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1,1444-10-22,1444-10-11,1444-10-14
4,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1,1444-10-22,1444-10-11,1444-10-15
5,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1,1444-10-22,1444-10-11,1444-10-16
6,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1,1444-10-22,1444-10-11,1444-10-17
7,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1,1444-10-22,1444-10-11,1444-10-18
8,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1,1444-10-22,1444-10-11,1444-10-19
9,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1,1444-10-22,1444-10-11,1444-10-20
